In [1]:
import os
import glob
import numpy as np
import random
import tensorflow as tf
from tensorflow import keras
from PIL import Image
from PIL import ImageEnhance

def transpose_image(path):
    image_dir_path = path
    images = glob.glob(image_dir_path + "/*.jpg")
    count = 0
    for img in images:
        sn = random.choice([0, 1, 2])
        if sn == 0:
            old_img = Image.open(img)
            new_img = old_img.transpose(Image.ROTATE_90)
            new_img.save("{}/r90_{}.jpg".format(image_dir_path, count), "JPEG")
        if sn == 1:
            old_img = Image.open(img)
            new_img = old_img.transpose(Image.ROTATE_180)
            new_img.save("{}/r180_{}.jpg".format(image_dir_path, count), "JPEG")
        if sn == 2:
            old_img = Image.open(img)
            new_img = old_img.transpose(Image.ROTATE_270)
            new_img.save("{}/r270_{}.jpg".format(image_dir_path, count), "JPEG")
        count+=1


def sharpness_image(path):
    image_dir_path = path
    images = glob.glob(image_dir_path + "/*.jpg")
    count = 0
    for img in images:
        old_img = Image.open(img)
        enhancer = ImageEnhance.Sharpness(old_img)
        new_img = enhancer.enhance(random.choice([0.5, 0.75, 1.5, 2]))
        new_img.save("{}/s_{}.jpg".format(image_dir_path, count), "JPEG")
        count+=1


def brightness_image(path):
    image_dir_path = path
    images = glob.glob(image_dir_path + "/*.jpg")
    count = 0
    for img in images:
        old_img = Image.open(img)
        enhancer = ImageEnhance.Brightness(old_img)
        new_img = enhancer.enhance(random.choice([0.5, 0.75, 1.5, 2]))
        new_img.save("{}/b_{}.jpg".format(image_dir_path, count), "JPEG")
        count+=1


def contrast_image(path):
    image_dir_path = path
    images = glob.glob(image_dir_path + "/*.jpg")
    count = 0
    for img in images:
        old_img = Image.open(img)
        enhancer = ImageEnhance.Contrast(old_img)
        new_img = enhancer.enhance(random.choice([0.5, 0.75, 1.5, 2]))
        new_img.save("{}/c_{}.jpg".format(image_dir_path, count), "JPEG")
        count+=1       


def resize_image(path):
    #image_dir_path = os.getenv("HOME") + path
    image_dir_path = path
    images = glob.glob(image_dir_path + "/*.jpg")  
    target_size=(28, 28)
    for img in images:
        old_img = Image.open(img)
        new_img = old_img.resize(target_size, Image.ANTIALIAS)
        new_img.save(img, "JPEG")        

이미지에 변화를 주어 데이터셋에 추가하는 과정을 진행해보기 위해 만들었던 함수
정확도 향상에 크게 기여하진 않지만 조금의 향상된 결과를 볼 수 있었다.

image_dir_path = os.getenv("HOME") + "/aiffel/rsp3/rock"
resize_image(image_dir_path)
image_dir_path = os.getenv("HOME") + "/aiffel/rsp3/scissor"
resize_image(image_dir_path)
image_dir_path = os.getenv("HOME") + "/aiffel/rsp3/paper"
resize_image(image_dir_path)

image_dir_path = os.getenv("HOME") + "/aiffel/rsp2/rock"
transpose_image(image_dir_path)
image_dir_path = os.getenv("HOME") + "/aiffel/rsp2/scissor"
transpose_image(image_dir_path)
image_dir_path = os.getenv("HOME") + "/aiffel/rsp2/paper"
transpose_image(image_dir_path)

image_dir_path = os.getenv("HOME") + "/aiffel/rsp2/rock"
contrast_image(image_dir_path)
image_dir_path = os.getenv("HOME") + "/aiffel/rsp2/scissor"
contrast_image(image_dir_path)
image_dir_path = os.getenv("HOME") + "/aiffel/rsp2/paper"
contrast_image(image_dir_path)

image_dir_path = os.getenv("HOME") + "/aiffel/rsp2/rock"
brightness_image(image_dir_path)
image_dir_path = os.getenv("HOME") + "/aiffel/rsp2/scissor"
brightness_image(image_dir_path)
image_dir_path = os.getenv("HOME") + "/aiffel/rsp2/paper"
brightness_image(image_dir_path)

image_dir_path = os.getenv("HOME") + "/aiffel/rsp2/rock"
sharpness_image(image_dir_path)
image_dir_path = os.getenv("HOME") + "/aiffel/rsp2/scissor"
sharpness_image(image_dir_path)
image_dir_path = os.getenv("HOME") + "/aiffel/rsp2/paper"
sharpness_image(image_dir_path)

In [2]:
def load_data(img_path, number_of_data):
    img_size = 28
    color = 3
    
    imgs = np.zeros(number_of_data * img_size * img_size * color, dtype = np.int32).reshape(number_of_data, img_size, img_size, color)
    labels = np.zeros(number_of_data, dtype = np.int32)

    idx = 0
    for file in glob.iglob(img_path + '/scissor/*.jpg'):
        img = np.array(Image.open(file), dtype = np.int32)
        imgs[idx, :, :, :] = img
        labels[idx] = 0
        idx = idx + 1

    for file in glob.iglob(img_path + '/rock/*.jpg'):
        img = np.array(Image.open(file), dtype = np.int32)
        imgs[idx, :, :, :] = img
        labels[idx] = 1
        idx = idx + 1       
    
    for file in glob.iglob(img_path + '/paper/*.jpg'):
        img = np.array(Image.open(file), dtype = np.int32)
        imgs[idx, :, :, :] = img
        labels[idx] = 2
        idx = idx + 1
        
    return imgs, labels

In [3]:
n_conv1=128
n_conv2=256
n_conv3=256
n_dense=64
n_epoch=15

model = keras.models.Sequential()
model.add(keras.layers.Conv2D(n_conv1, (3, 3), activation = 'relu', input_shape=(28, 28, 3)))
model.add(keras.layers.MaxPooling2D(2, 2))
model.add(keras.layers.Conv2D(n_conv2, (3, 3), activation = 'relu'))
model.add(keras.layers.MaxPooling2D(2, 2))
model.add(keras.layers.Conv2D(n_conv3, (3, 3), activation = 'relu'))
model.add(keras.layers.MaxPooling2D(2, 2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(n_dense, activation = 'relu'))
model.add(keras.layers.Dense(3, activation = 'softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 128)       3584      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 256)       295168    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 256)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 256)         590080    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 1, 1, 256)         0         
_________________________________________________________________
flatten (Flatten)            (None, 256)               0

Param # 가중치와 편향 개수

image_dir_path = os.getenv("HOME") + "/aiffel/rsp"
(x_train, y_train)=load_data(image_dir_path, 3900)
x_train_norm = x_train/255.0

In [4]:
from sklearn.model_selection import train_test_split
image_dir_path = os.getenv("HOME") + "/aiffel/rsp"
(X, y)=load_data(image_dir_path, 3900)
x_train,x_test,y_train,y_test = train_test_split(X,y, test_size=0.2, random_state=1)
x_train_norm = x_train/255.0 #normalization
x_test_norm = x_test/255.0

In [5]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train_norm, y_train, epochs = n_epoch)

Epoch 1/15
98/98 [==============================] - 10s 51ms/step - loss: 1.1031 - accuracy: 0.3493
Epoch 2/15
98/98 [==============================] - 1s 9ms/step - loss: 1.0325 - accuracy: 0.4456
Epoch 3/15
98/98 [==============================] - 1s 9ms/step - loss: 0.6530 - accuracy: 0.7150
Epoch 4/15
98/98 [==============================] - 1s 10ms/step - loss: 0.4288 - accuracy: 0.8406
Epoch 5/15
98/98 [==============================] - 1s 10ms/step - loss: 0.2939 - accuracy: 0.8896
Epoch 6/15
98/98 [==============================] - 1s 10ms/step - loss: 0.1953 - accuracy: 0.9244
Epoch 7/15
98/98 [==============================] - 1s 10ms/step - loss: 0.1016 - accuracy: 0.9658
Epoch 8/15
98/98 [==============================] - 1s 10ms/step - loss: 0.0539 - accuracy: 0.9840
Epoch 9/15
98/98 [==============================] - 1s 10ms/step - loss: 0.0656 - accuracy: 0.9803
Epoch 10/15
98/98 [==============================] - 1s 10ms/step - loss: 0.0647 - accuracy: 0.9765
Epoch 11/1

In [6]:
image_dir_path = os.getenv("HOME") + "/aiffel/rsp" #train_test_split 함수를 이용한 테스트데이터
(X, y)=load_data(image_dir_path, 3900)
x_train,x_test,y_train,y_test = train_test_split(X,y, test_size=0.2, random_state=1)
x_train_norm = x_train/255.0
x_test_norm = x_test/255.0

In [7]:
test_loss, test_accuracy = model.evaluate(x_test_norm, y_test, verbose = 2)
print("loss : {}".format(test_loss))
print("accuracy : {}".format(test_accuracy))

25/25 - 2s - loss: 0.0167 - accuracy: 0.9936
loss : 0.016730744391679764
accuracy : 0.9935897588729858


In [8]:
image_dir_path = os.getenv("HOME") + "/aiffel/rsp/test" #학습 전 데이터셋에서 직접 골라낸 이미지 테스트데이터 300개
(x_test, y_test)=load_data(image_dir_path, 300)
x_test_norm = x_test/255.0

In [9]:
test_loss, test_accuracy = model.evaluate(x_test_norm, y_test, verbose = 2)
print("loss : {}".format(test_loss))
print("accuracy : {}".format(test_accuracy))

10/10 - 0s - loss: 0.0232 - accuracy: 0.9900
loss : 0.023226678371429443
accuracy : 0.9900000095367432


In [10]:
image_dir_path = os.getenv("HOME") + "/aiffel/rsp3" #새로운 데이터셋
(x_test, y_test)=load_data(image_dir_path, 800)
x_test_norm = x_test/255.0

In [11]:
test_loss, test_accuracy = model.evaluate(x_test_norm, y_test, verbose = 2)
print("loss : {}".format(test_loss))
print("accuracy : {}".format(test_accuracy))

25/25 - 0s - loss: 2.6241 - accuracy: 0.7513
loss : 2.624124050140381
accuracy : 0.7512500286102295


In [12]:
image_dir_path = os.getenv("HOME") + "/aiffel/rsp3" #새로운 데이터셋에서 train_test_split으로 240개 테스트데이터 선택
(X, y)=load_data(image_dir_path, 800)
x_train,x_test,y_train,y_test = train_test_split(X,y, test_size=0.3)
x_train_norm = x_train/255.0
x_test_norm = x_test/255.0
test_loss, test_accuracy = model.evaluate(x_test_norm, y_test, verbose = 2)
print("loss : {}".format(test_loss))
print("accuracy : {}".format(test_accuracy))

8/8 - 0s - loss: 2.5999 - accuracy: 0.7417
loss : 2.599865436553955
accuracy : 0.7416666746139526


conv1 128
conv2 256
conv3 256
dense 64
epoch 15
loss: 0.0281 - accuracy: 0.9962 loss: 0.0143 - accuracy: 0.9933 loss: 1.6875 - accuracy: 0.7887 loss: 1.3083 - accuracy: 0.8333
loss: 0.0269 - accuracy: 0.9949 loss: 0.0377 - accuracy: 0.9900 loss: 2.3357 - accuracy: 0.7325 loss: 2.0729 - accuracy: 0.7458
loss: 0.0496 - accuracy: 0.9859 loss: 0.0485 - accuracy: 0.9800 loss: 2.2167 - accuracy: 0.7875 loss: 1.9866 - accuracy: 0.8083

conv1 128
conv2 256
conv3 256
dense 64
epoch 30
loss: 0.0262 - accuracy: 0.9962 loss: 0.0069 - accuracy: 0.9967 loss: 2.2958 - accuracy: 0.7800 loss: 2.2688 - accuracy: 0.7792
loss: 0.0144 - accuracy: 0.9962 loss: 0.0272 - accuracy: 0.9933 loss: 3.5359 - accuracy: 0.7588 loss: 3.4498 - accuracy: 0.7708
loss: 0.0031 - accuracy: 0.9987 loss: 0.0352 - accuracy: 0.9933 loss: 2.3547 - accuracy: 0.8025 loss: 2.6465 - accuracy: 0.8083

conv1 32
conv2 64
conv3 128
dense 64
epoch 15
loss: 0.0542 - accuracy: 0.9897 loss: 0.0340 - accuracy: 0.9833 loss: 1.8093 - accuracy: 0.7450 loss: 2.1536 - accuracy: 0.7000
loss: 0.0861 - accuracy: 0.9731 loss: 0.1016 - accuracy: 0.9567 loss: 1.0858 - accuracy: 0.7950 loss: 0.8624 - accuracy: 0.8125
loss: 0.0672 - accuracy: 0.9821 loss: 0.0340 - accuracy: 0.9967 loss: 2.0408 - accuracy: 0.6562 loss: 1.7934 - accuracy: 0.6792

conv1 512
conv2 1024
conv3 1024
dense 512
epoch 15
loss: 0.0886 - accuracy: 0.9769 loss: 0.0543 - accuracy: 0.9833 loss: 2.6290 - accuracy: 0.6875 loss: 2.5726 - accuracy: 0.6708
loss: 0.0382 - accuracy: 0.9936 loss: 0.0369 - accuracy: 0.9867 loss: 2.6970 - accuracy: 0.7475 loss: 2.8757 - accuracy: 0.7208
loss: 0.0552 - accuracy: 0.9846 loss: 0.0331 - accuracy: 0.9867 loss: 3.4652 - accuracy: 0.6375 loss: 3.5575 - accuracy: 0.6583


normalization x
conv1 128
conv2 256
conv3 256
dense 64
epoch 15
loss: 0.1763 - accuracy: 0.9372 loss: 0.2103 - accuracy: 0.9100 loss: 3.1583 - accuracy: 0.5863 loss: 3.5713 - accuracy: 0.5750
epoch 30
loss: 0.1041 - accuracy: 0.9910 loss: 0.0264 - accuracy: 0.9900 loss: 2.1943 - accuracy: 0.7312 loss: 2.3874 - accuracy: 0.7250
epoch 40
loss: 0.0084 - accuracy: 0.9949 loss: 0.0402 - accuracy: 0.9933 loss: 3.3139 - accuracy: 0.7550 loss: 2.5302 - accuracy: 0.7875


conv1 128
conv2 256
dense 64
epoch 15
loss: 0.0696 - accuracy: 0.9859 loss: 0.0278 - accuracy: 0.9933 loss: 2.0714 - accuracy: 0.6725 loss: 2.0650 - accuracy: 0.6708
epoch 30
loss: 0.0522 - accuracy: 0.9974 loss: 0.0163 - accuracy: 0.9967 loss: 3.0442 - accuracy: 0.6762 loss: 2.7411 - accuracy: 0.7083
epoch 50
loss: 0.0631 - accuracy: 0.9962 loss: 0.0037 - accuracy: 1.0000 loss: 4.2513 - accuracy: 0.7025 loss: 5.3597 - accuracy: 0.6667

conv1 128
dense 64
epoch 15
loss: 0.0757 - accuracy: 0.9872 loss: 0.0358 - accuracy: 0.9933 loss: 2.6967 - accuracy: 0.5788 loss: 2.8090 - accuracy: 0.6000
epoch 30
loss: 0.0479 - accuracy: 0.9962 loss: 0.0172 - accuracy: 0.9967 loss: 3.8713 - accuracy: 0.5612 loss: 3.9520 - accuracy: 0.5500
epoch 5
loss: 0.2757 - accuracy: 0.9090 loss: 0.2668 - accuracy: 0.8933 loss: 1.4980 - accuracy: 0.4988 loss: 1.4372 - accuracy: 0.5167

하이퍼파라미터 값에 따라 결과가 조금씩 달라지긴 하지만 큰 변화를 보이지는 않았다.
적당한 하이퍼파라미터로 이미지에서 특징추출을 해주어야 한다. 너무 작거나 크면 특징추출이 잘 안되고 하이퍼파라미터가 클수록 연산량이 많아서 학습시간이 길어지기 때문에 100 ~ 200 근처의 값이 적당한 값이라 한다.

정규화를 통해 입력 데이터를 유사한 분포를 가지도록 하는 것이 좋다. 정규화는 학습과정에서 수렴을 더 쉽고 빠르게 만들어준다.
정규화를 했을 때 epoch 15로 충분했지만 정규화를 하지 않았을 때는 epoch 30 ~ 40 정도로 높여야 비슷한 결과를 볼 수 있었다.

레이어 개수를 1 ~ 3개 바꾸어 보았을 때 하나의 데이터셋을 이용 할 때는 큰 차이가 없지만 새로운 데이터셋을 이용하여 테스트 해 보았을 때 레이어 개수가 많을수록 정확도가 높게 나타났다. 레이어 개수가 많을수록 성능이 높아지지만 너무 많으면 의사결정을 하지 못하는 경우가 생길 수 있다고 한다.

동일한 데이터셋, 파라미터 사용하여 반복 수행해도 정확도가 조금씩 달라진다.
model.fit() 함수에 기본값으로 shuffle=True 설정이 되어있어서 데이터가 순서나 특정 규칙에 따라 분포하는 경우 이러한 특성이 학습될 수 있기 때문에 shuffle을 통해 분포가 고르게 만들어 학습 성능을 높인다.

모델을 학습하고 시험하는데 있어 가장 중요한 것은 데이터셋이다. 데이터셋이 다양하게 많을수록 좋은 모델로 학습시키고 시험성능이 향상된다.
train_test_split에서 테스트사이즈를 증가시키면 시험성능이 낮아졌다. 학습량도 줄고 학습하지 못한 테스트데이터도 늘어나다보니 그런것같다. 학습데이터와 다른 데이터셋을 이용해 시험할 경우 시험데이터에 따라 정확도가 크게 차이났다. 좀 더 크고 화질이 좋은 이미지로 학습데이터와 모델을 개선한다면 새로운 시험 데이터에 대한 정확도도 좋아지겠지만, 이번 과제에서는 28x28 크기의 작은 이미지를 이용하여 구분하기 힘든 데이터도 있었고, 데이터 분포가 크지않은 데이터셋을 이용하였다. 그러다 보니 배경이나 밝기 등 주변 환경이 생소한 시험데이터의 경우 정확도가 많이 떨어졌다.